In [1]:
import simpy
import numpy as np

#RANDOM_SEEDS = np.random.uniform(0, 1000, 10)
#RANDOM_SEEDS = list(map(int, RANDOM_SEEDS))
RANDOM_SEEDS = [0, 194, 482, 241, 832, 997, 352, 153, 56, 569]
TOTAL_CUSTOMER = 0   # We are gonna change it to 5000 after applying 10 different seed.

INTERARRIVAL_RATE = 6
MACHINE_WAIT_RATE = 5
SHIFT_TIME = 8*60
SIM_OVER = False

N_MEAN = np.log(12/(np.sqrt(1 + ((6**2)/(12**2)))))
N_STD = np.sqrt(np.log(1 + ((6**2)/(12**2))))

num_customers_unsatisfied = 0
num_waiting_customer = [0, 0]
system_end_time = 0
answering_times = [] #Duration of the conversation between the customer and the anwering system. (Info collecting time)
service_times = [[],[]] #Duration of the conversation between the customer and the operator (Service time)
queue_w_times = [[],[]] #Time spent by a customer while it waits for the operator (Queue waiting time Wq)
customer_total_system_t = []

answering_utils = []
operators_utils = [[], []]
ave_total_waiting_times = []
max_total_w_system_time_ratios = []
ave_people_w_OP0s = []
ave_people_w_OP1s = []
num_people_unsatisfieds = []

In [2]:
class Customer(object):
    
    def __init__(self, name, env, arrival_t, operator):
        self.env = env
        self.name = name
        self.operator = operator
        self.arrival_t = arrival_t
        self.departure_t = float('inf')
        self.action = self.env.process(self.call())
        

    def call(self):
        operator_no = int(np.random.random() > 0.3)
        correct_operator = np.random.random() > 0.1
        global system_end_time, num_customers_unsatisfied, num_waiting_customer
        if correct_operator:
            current_operator = self.operator[operator_no]
            with current_operator.resource.request() as request:
                num_waiting_customer[operator_no] += 1
                self.entering_queue_t = self.env.now
                patience = self.env.timeout(10)
                yield request | patience
                waiting_t_queue = self.env.now - self.entering_queue_t
                if(waiting_t_queue == 0):
                    num_waiting_customer[operator_no] -= 1
            
                queue_w_times[operator_no].append(waiting_t_queue)
                if request.triggered:
                    yield self.env.process(current_operator.serve())
                    self.departure_t = self.env.now
                else:
                    num_customers_unsatisfied += 1
                
        else:
            queue_w_times[operator_no].append(0) 
            num_customers_unsatisfied += 1
            
        customer_total_system_t.append(self.env.now - self.arrival_t)
            
        system_end_time = self.env.now
        if(len(queue_w_times[0]) + len(queue_w_times[1]) == TOTAL_CUSTOMER):
            SIM_OVER = True

In [3]:
class Operator(object):

    def __init__(self, env, name, res):
        self.resource = res
        self.env = env
        self.name = name
        self.breaks_to_take = 0
        self.breaks_from_prev = 0
        self.number_of_breaks = np.random.poisson(8)
        self.env.process(self.schedule_break())
        self.env.process(self.shift_change())
       
    
    def take_break(self):
        flag = False
        if(SIM_OVER):
            return
        with self.resource.request() as request:
            yield request
            if(SIM_OVER):
                return
            if(self.breaks_from_prev > 0 ):
                self.breaks_from_prev -= 1
                flag = True
            else:
                if len(self.resource.queue) == self.breaks_to_take - 1:
                    self.breaks_to_take -= 1
                    remaining_shift_time = (8 * 60) - (self.env.now % (8 * 60))
                    if(remaining_shift_time < 3):
                        yield self.env.timeout(remaining_shift_time)
                    else:
                        yield self.env.timeout(3)
                    flag = True

        if not flag:
            yield self.env.process(self.take_break())
    
    
    def serve(self):
        global service_times
        if self.name == 0:
            service_time = np.random.lognormal(N_MEAN,N_STD)
        else:
            service_time = np.random.uniform(1,7)
        service_times[self.name].append(service_time)
        yield self.env.timeout(service_time)
        
        
    def schedule_break(self):
        break_times = sorted(np.random.uniform(0, 8*60, self.number_of_breaks))
        for break_time in break_times:
            if(SIM_OVER):
                return
            duration = break_time - (self.env.now % (8 * 60))
            duration = duration if duration > 0 else 0
            yield self.env.timeout(duration)
            self.breaks_to_take += 1
            self.env.process(self.take_break())            

            
    def shift_change(self):
        while not SIM_OVER:
            yield self.env.timeout(8*60)
            if(SIM_OVER):
                return
            self.breaks_from_prev = self.breaks_to_take
            self.breaks_to_take = 0
            self.number_of_breaks = np.random.poisson(8)
            self.env.process(self.schedule_break())

In [4]:
class AnsweringSystem(object):
    
    def __init__(self, env, counter, operators, machine):
        self.env = env
        self.counter = counter
        self.operators = operators
        self.machine = machine
        self.action = self.env.process(self.run())
        
        
    def run(self):
        with self.machine.request() as request:
            yield request
            arrival_t = self.env.now
            duration = np.random.exponential(MACHINE_WAIT_RATE)
            answering_times.append(duration)   # We are gonna use it to find utilization of answering system
            yield self.env.timeout(duration)
            Customer('Cust %s' %(self.counter), self.env, arrival_t, self.operators)

In [5]:
def customer_generator(env, operators, machine):
    global SIM_OVER
    counter = 0
    while(counter < TOTAL_CUSTOMER):
        yield env.timeout(np.random.exponential(INTERARRIVAL_RATE))
        if machine.count < 100:
            counter += 1
            AnsweringSystem(env, counter, operators, machine)
    SIM_OVER = True 

In [6]:
def reset_global_variables():
    global num_customers_unsatisfied, num_waiting_customer
    global system_end_time,service_times,queue_w_times,customer_total_system_t
    
    num_customers_unsatisfied = 0
    num_waiting_customer = [0, 0]
    system_end_time = 0
    answering_times = [] 
    service_times = [[],[]] 
    queue_w_times = [[],[]] 
    customer_total_system_t = []
    num_people_unsatisfied = 0
    
    
def simulate(seed):
    np.random.seed(seed)
    env = simpy.Environment()
    operator_queues = [simpy.Resource(env, capacity = 1), simpy.Resource(env, capacity = 1)]
    operators = [Operator(env, 0, operator_queues[0]), Operator(env, 1, operator_queues[1])]
    machine = simpy.Resource(env, capacity = 100)
    env.process(customer_generator(env, operators, machine))
    env.run()

    
def collect_data():
    global num_customers_unsatisfied, num_waiting_customer
    global system_end_time,service_times,queue_w_times,customer_total_system_t
    
    #1 Utilization of the answering system
    total_answering_time = sum(answering_times)
    answering_utils.append((total_answering_time / system_end_time)/ 100 ) 

    #2 Utilization of the operators  
    OP0_util = sum(service_times[0]) / system_end_time
    OP1_util = sum(service_times[1]) / system_end_time 
    operators_utils[0].append(OP0_util)
    operators_utils[1].append(OP1_util)

    #3 Average Total Waiting Time
    ave_total_waiting_time = (sum(queue_w_times[0]) + sum(queue_w_times[1])) / TOTAL_CUSTOMER
    ave_total_waiting_times.append(ave_total_waiting_time)

    #4 Maximum Total Waiting Time to Total System Time Ratio
    max_total_w_system_time_ratio = (sum(queue_w_times[0]) + sum(queue_w_times[1]))/sum(customer_total_system_t)
    max_total_w_system_time_ratios.append(max_total_w_system_time_ratio)

    #5 Average number of people waiting to be served by each operator
    ave_people_w_OP0 = sum(queue_w_times[0]) / system_end_time
    ave_people_w_OP1 = sum(queue_w_times[1]) / system_end_time
    ave_people_w_OP0s.append(ave_people_w_OP0)
    ave_people_w_OP1s.append(ave_people_w_OP1)

    #6 Average number of customers leaving the system unsatisfied either due to incorrect routing or due to long waiting times.
    num_people_unsatisfied = num_customers_unsatisfied
    num_people_unsatisfieds.append(num_people_unsatisfied)

def print_statistics():
    for i in range(0,10):
        print(f"###########")
        print(f"Simulation {i+1} with {TOTAL_CUSTOMER} customers.")
        print(f"###########")
        print(f"Utilization of the answering system: {answering_utils[i]}")
        print(f"Utilization of the operator 0: {operators_utils[0][i]}")
        print(f"Utilization of the operator 1: {operators_utils[1][i]}")
        print(f"Average Total Waiting Time: {ave_total_waiting_times[i]}")
        print(f"Maximum Total Waiting Time to Total System Time Ratio: {max_total_w_system_time_ratios[i]}")
        print(f"Average number of people waiting to be served by the operator 0: {ave_people_w_OP0s[i]}")
        print(f"Average number of people waiting to be served by the operator 1: {ave_people_w_OP1s[i]}")
        print(f"Number of customers leaving the system unsatisfied: {num_people_unsatisfieds[i]}")
    print(f"###########")
    print(f"General results of simulations with {TOTAL_CUSTOMER} customers.")
    print(f"###########")
    print(f"Means".center(15,"-"))
    print(f"Utilization of the answering system: {np.mean(answering_utils)}")
    print(f"Utilization of the operator 0: {np.mean(operators_utils[0])}")
    print(f"Utilization of the operator 1: {np.mean(operators_utils[1])}")
    print(f"Average Total Waiting Time: {np.mean(ave_total_waiting_times)}")
    print(f"Maximum Total Waiting Time to Total System Time Ratio: {np.mean(max_total_w_system_time_ratios)}")
    print(f"Average number of people waiting to be served by the operator 0: {np.mean(ave_people_w_OP0s)}")
    print(f"Average number of people waiting to be served by the operator 1: {np.mean(ave_people_w_OP1s)}")
    print(f"Variances".center(15,"-"))
    print(f"Utilization of the answering system: {np.var(answering_utils)}")
    print(f"Utilization of the operator 0: {np.var(operators_utils[0])}")
    print(f"Utilization of the operator 1: {np.var(operators_utils[1])}")
    print(f"Average Total Waiting Time: {np.var(ave_total_waiting_times)}")
    print(f"Maximum Total Waiting Time to Total System Time Ratio: {np.var(max_total_w_system_time_ratios)}")
    print(f"Average number of people waiting to be served by the operator 0: {np.var(ave_people_w_OP0s)}")
    print(f"Average number of people waiting to be served by the operator 1: {np.var(ave_people_w_OP1s)}")
    print(f"Others".center(15,"-"))
    print(f"Average number of customers leaving the system unsatisfied (for 1000 customers): {sum(num_people_unsatisfieds)/len(num_people_unsatisfieds)}")

In [7]:
answering_utils = []
operators_utils = [[], []]
ave_total_waiting_times = []
max_total_w_system_time_ratios = []
ave_people_w_OP0s = []
ave_people_w_OP1s = []
num_people_unsatisfieds = []

TOTAL_CUSTOMER = 1000
for seed in RANDOM_SEEDS:
    reset_global_variables()
    simulate(seed)
    collect_data()
print_statistics()

###########
Simulation 1 with 1000 customers.
###########
Utilization of the answering system: 0.008033934698879452
Utilization of the operator 0: 0.4770058440650503
Utilization of the operator 1: 0.3898222629010887
Average Total Waiting Time: 1.8439275281934189
Maximum Total Waiting Time to Total System Time Ratio: 0.15242145050602074
Average number of people waiting to be served by the operator 0: 0.14502247233692525
Average number of people waiting to be served by the operator 1: 0.15533623254363113
Number of customers leaving the system unsatisfied: 153
###########
Simulation 2 with 1000 customers.
###########
Utilization of the answering system: 0.01624086973354822
Utilization of the operator 0: 0.4467955570341096
Utilization of the operator 1: 0.4196633541650185
Average Total Waiting Time: 1.7901401499336975
Maximum Total Waiting Time to Total System Time Ratio: 0.14804426682848051
Average number of people waiting to be served by the operator 0: 0.1375493831550975
Average number 

In [8]:
answering_utils = []
operators_utils = [[], []]
ave_total_waiting_times = []
max_total_w_system_time_ratios = []
ave_people_w_OP0s = []
ave_people_w_OP1s = []
num_people_unsatisfieds = []
    
TOTAL_CUSTOMER = 5000
for seed in RANDOM_SEEDS:
    reset_global_variables()
    simulate(seed)
    collect_data()   
print_statistics()

###########
Simulation 1 with 5000 customers.
###########
Utilization of the answering system: 0.02546129775996787
Utilization of the operator 0: 0.4603565122440308
Utilization of the operator 1: 0.41410401592443813
Average Total Waiting Time: 1.7981712713816145
Maximum Total Waiting Time to Total System Time Ratio: 0.15094589214111315
Average number of people waiting to be served by the operator 0: 0.13770176338476992
Average number of people waiting to be served by the operator 1: 0.166859832424807
Number of customers leaving the system unsatisfied: 792
###########
Simulation 2 with 5000 customers.
###########
Utilization of the answering system: 0.03367904854828251
Utilization of the operator 0: 0.45891675068121196
Utilization of the operator 1: 0.4195596114491836
Average Total Waiting Time: 1.8901394963420046
Maximum Total Waiting Time to Total System Time Ratio: 0.15720238768651806
Average number of people waiting to be served by the operator 0: 0.14932632532821333
Average number 